In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

# Biomedical Core (electives already included in design and technical electives list)

## 1. Collect course codes

In [2]:
url = "http://www.calendar.ubc.ca/okanagan/index.cfm?tree=18,317,989,1382"
uClient = ureq(url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")

In [5]:
containers = page_soup.find("h3", text = "Biomedical Option").findNext("ul", {"class": "double"})
containers

<ul class="double"><li>APSC 193 Anatomy and Physiology for Engineers
<li>Required 4th year courses (as listed above) and:
<ul>
<li>ENGR 351 Microelectronics I
<li>ENGR 401 Bioinstrumentation
<li>ENGR 402 Biotechnology: Fundamentals and Applications
<li>ENGR 406 Microelectromechanical Systems
<li>ENGR 423 Wearables
<li>ENGR 450 Clinical Engineering
<li>ENGR 482 Biomedical Engineering I
<li>ENGR 499 Engineering Capstone Project – Biomedical Project
</li></li></li></li></li></li></li></li></ul></li></li></ul>

In [12]:
containers.text.split("\r\n")

['APSC 193 Anatomy and Physiology for Engineers',
 'Required 4th year courses (as listed above) and:',
 '\nENGR 351 Microelectronics I',
 'ENGR 401 Bioinstrumentation',
 'ENGR 402 Biotechnology: Fundamentals and Applications',
 'ENGR 406 Microelectromechanical Systems',
 'ENGR 423 Wearables',
 'ENGR 450 Clinical Engineering',
 'ENGR 482 Biomedical Engineering I',
 'ENGR 499 Engineering Capstone Project – Biomedical Project',
 '']

In [14]:
course_codes = [code.strip() for code in containers.text.split("\r\n") if len(code) > 1 and bool(re.match("^[A-Z]{4}", code.strip()))]
course_codes

['APSC 193 Anatomy and Physiology for Engineers',
 'ENGR 351 Microelectronics I',
 'ENGR 401 Bioinstrumentation',
 'ENGR 402 Biotechnology: Fundamentals and Applications',
 'ENGR 406 Microelectromechanical Systems',
 'ENGR 423 Wearables',
 'ENGR 450 Clinical Engineering',
 'ENGR 482 Biomedical Engineering I',
 'ENGR 499 Engineering Capstone Project – Biomedical Project']

In [17]:
course_codes = [" ".join(code.split()[:2]) for code in course_codes][:-1]
course_codes

['APSC 193',
 'ENGR 351',
 'ENGR 401',
 'ENGR 402',
 'ENGR 406',
 'ENGR 423',
 'ENGR 450',
 'ENGR 482',
 'ENGR 499']

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [18]:
subject_links = {
    "ENGR": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=ENGR",
    "MANF": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=MANF",
    "APSC": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=APSC"
}

In [19]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  APSC 193
scraped  ENGR 351
scraped  ENGR 401
scraped  ENGR 402
scraped  ENGR 406
scraped  ENGR 423
scraped  ENGR 450
scraped  ENGR 482
scraped  ENGR 499
9


## 3. Inspect and write to CSV

In [20]:
course_names

['Anatomy and Physiology for Engineers',
 'Microelectronics I',
 'Bioinstrumentation',
 'Biotechnology: Fundamentals and Applications',
 'Microelectromechanical Systems',
 'Wearable Devices',
 'Clinical Engineering',
 'Biomedical Engineering I',
 'Engineering Capstone Design Project']

In [22]:
course_descs

['Structure and function of the neuromuscular, skeletal, digestive, endocrine, urinary, circulatory, and respiratory systems of the human body. Special emphasis on interfacing with biomedical devices. [3-2-0]',
 'Signals and amplifier fundamentals, the operational amplifier, diodes, metal-oxide-semiconductor field effect transistor amplifier circuits, and bipolar junction transistor amplifier circuits. [3-2*-0] Prerequisite: APSC 255.',
 'Bioinstruments used for tracking vitals, diagnosis, and treatment of disease in the vascular, muscular, nervous, and respiratory systems. Introduction to the fundamentals of each body system, electrical safety, signal acquisition, biosensors, transducers, amplifiers, and analysis of human physiological measurements. Hands on experience with sensors, biomedical devices, and design through labs. [3-2*-0] Prerequisite: All of APSC 193, APSC 254.',
 'Basics of biotechnology, DNA and RNA technologies, nanobiotechnology, medical biotechnology, and ethics in

In [23]:
import pandas as pd
df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,APSC 193,Anatomy and Physiology for Engineers,"Structure and function of the neuromuscular, s..."
1,ENGR 351,Microelectronics I,"Signals and amplifier fundamentals, the operat..."
2,ENGR 401,Bioinstrumentation,"Bioinstruments used for tracking vitals, diagn..."
3,ENGR 402,Biotechnology: Fundamentals and Applications,"Basics of biotechnology, DNA and RNA technolog..."
4,ENGR 406,Microelectromechanical Systems,"Mechanisms, design, fabrication, and testing o..."
5,ENGR 423,Wearable Devices,"Design and prototyping including textiles, har..."
6,ENGR 450,Clinical Engineering,"The clinical environment, clinical tools (for ..."
7,ENGR 482,Biomedical Engineering I,Introduction to the microcirculation; gas exch...
8,ENGR 499,Engineering Capstone Design Project,A capstone design project in response to an ac...


In [24]:
df.to_csv('UBC_Okanagan_MechEng_Biomedical_Option_Courses.csv', index = False)

# Mechatronics Core and Electives

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

## 1. Collect course codes

In [2]:
import pandas as pd
df = pd.read_excel(r'UBC Okanagan MechEng Mechatronics Option Course Names.xlsx', header = None)
df

,0
0,ENGR 406
1,ENGR 458
2,ENGR 454
3,ENGR 467
4,ENGR 418
5,ENGR 453
6,ENGR 486
7,ENGR 487
8,ENGR 456
9,ENGR 494


In [3]:
course_codes = list(df[0])
course_codes

['ENGR 406',
 'ENGR 458',
 'ENGR 454',
 'ENGR 467',
 'ENGR 418',
 'ENGR 453',
 'ENGR 486',
 'ENGR 487',
 'ENGR 456',
 'ENGR 494',
 'COSC 121',
 'COSC 222',
 'MANF 386',
 'ENGR 359',
 'ENGR 480',
 'MANF 486']

## 2. Use course codes to find and scrape corresponding course names and descriptions

In [4]:
subject_links = {
    "ENGR": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=ENGR",
    "MANF": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=2&code=MANF",
    "COSC": "http://www.calendar.ubc.ca/okanagan/courses.cfm?go=code&institution=8&code=COSC"
}

In [5]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  ENGR 406
scraped  ENGR 458
scraped  ENGR 454
scraped  ENGR 467
scraped  ENGR 418
scraped  ENGR 453
scraped  ENGR 486
scraped  ENGR 487
scraped  ENGR 456
scraped  ENGR 494
scraped  COSC 121
scraped  COSC 222
scraped  MANF 386
scraped  ENGR 359
scraped  ENGR 480
scraped  MANF 486
16


## 3. Inspect and write to CSV

In [6]:
course_names

['Microelectromechanical Systems',
 'Power Electronics',
 'Motor Drive Systems',
 'Real-Time and Embedded System Design',
 'Applied Machine Learning for Engineers',
 'Internet of Things',
 'Robot Modelling and Control',
 'Digital Control',
 'Electrochemical Energy Storage Systems',
 'Autonomous Vehicle Technology',
 'Computer Programming II',
 'Data Structures',
 'Industrial Automation',
 'Microcomputer Engineering',
 'Modern Control',
 'Mechatronic Systems Laboratory']

In [7]:
course_descs

['Mechanisms, design, fabrication, and testing of microsensors, actuators, and MicroElectroMechanical systems (MEMS).  Credit will not be granted for both ENGR 406 and ENGR 506 [3-2*-0] Prerequisite: Fourth-year standing in the B.A.Sc. Program',
 'Applications and roles of power electronics, power semiconductor devices, diode rectifiers, phase-controlled rectifiers, DC-DC converters, DC-AC converters, resonant converters. Examples drawn from residential and industrial applications. Credit will be granted for only one of ENGR 458 or ENGR 558. [3-2*-0] Prerequisite: ENGR 320.',
 'Three-phase AC/DC PWM inverter, converter modulation techniques, abc/qd reference frame theory, brushed DC machine drives, induction motor drives, permanent magnet AC machines, brushless dc motors and drive circuits. [3-2*-0] Prerequisite: ENGR 320.',
 'Multi-tasking, interrupt-driven systems, RTOSs and programming environments, task scheduling, schedulability analysis, inter-process communication and synchroniz

In [8]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,ENGR 406,Microelectromechanical Systems,"Mechanisms, design, fabrication, and testing o..."
1,ENGR 458,Power Electronics,"Applications and roles of power electronics, p..."
2,ENGR 454,Motor Drive Systems,"Three-phase AC/DC PWM inverter, converter modu..."
3,ENGR 467,Real-Time and Embedded System Design,"Multi-tasking, interrupt-driven systems, RTOSs..."
4,ENGR 418,Applied Machine Learning for Engineers,"Fundamentals of machine learning, toolboxes in..."
5,ENGR 453,Internet of Things,"Sensing, actuation, sampling, analog-to-digita..."
6,ENGR 486,Robot Modelling and Control,Spatial description and homogeneous transforma...
7,ENGR 487,Digital Control,Digital control theory and a brief review of c...
8,ENGR 456,Electrochemical Energy Storage Systems,Thermodynamics and kinetics of electrochemical...
9,ENGR 494,Autonomous Vehicle Technology,"Autonomous navigation: perception, localizatio..."


In [9]:
df.to_csv("UBC_Okanagan_MechEng_Mechatronics_Option_Courses.csv", index = False)